In [ ]:
import os
import urllib.request

url="https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/waimai_10k/waimai_10k.csv"
#設定儲存的檔案路徑及名稱
filepath="waimai_10k.csv"
# 判斷檔案是否存在，若不存在才下載
if not os.path.isfile(filepath):
    # 下載檔案
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [ ]:
import pandas as pd
pd_all = pd.read_csv('waimai_10k.csv')

print('評論數目（全部）：%d' % pd_all.shape[0])
print('評論數目（正面）：%d' % pd_all[pd_all.label==1].shape[0])
print('評論數目（負面）：%d' % pd_all[pd_all.label==0].shape[0])

評論數目（全部）：11987
評論數目（正面）：4000
評論數目（負面）：7987


In [ ]:
train_positive = pd_all[pd_all.label==1]['review']
train_negative = pd_all[pd_all.label==0]['review']

In [ ]:
import csv
import numpy as np
def read_files():
    path = 'waimai_10k.csv'
    label = []
    all_texts = []
    all_label = []
    #取得review資料
    with open(path, newline='') as csvfile_train:
        reader = csv.DictReader(csvfile_train)
        content = [row['review'] for row in reader]
        all_texts+=content
    #取得label資料
    with open(path, newline='') as csvfile_label:
        reader = csv.DictReader(csvfile_label)
        tag = [row['label'] for row in reader]
        label+=tag
    #將label list的值轉為int格式
    all_label = list(map(int, label))
    return all_texts,all_label

In [ ]:
train,label=read_files()
print(train[3999])
print(label[3999])
print(train[4000])
print(label[4000])

味道不错,配送速度比预计快
1
菜品质量好，味道好，就是百度的问题，总是用运力原因来解释，我也不懂这是什么原因，晚了三个小时呵呵厉害吧！反正订了就退不了，只能干等……
0


In [ ]:
train = train[:8000]
label = label[:8000]

In [ ]:
import random
x_shuffle=train
y_shuffle=label
z_shuffle = list(zip(x_shuffle, y_shuffle))

random.shuffle(z_shuffle)

x_train, y_label = zip(*z_shuffle)

In [ ]:
print(label[:10])
print(y_label[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(1, 1, 0, 0, 0, 0, 0, 0, 0, 1)


In [ ]:
# from keras.utils import np_utils
# y_label = np_utils.to_categorical(y_label, 2)

In [ ]:
NUM_TRAIN = int(8000 * 0.8)
train, test = x_train[:NUM_TRAIN], x_train[NUM_TRAIN:]
labels_train, labels_test = y_label[:NUM_TRAIN], y_label[NUM_TRAIN:]

In [ ]:
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

In [ ]:
stopWords=[]
with open('stopWord.txt', 'r', encoding='utf8') as f:
    stopWords = f.read().split('\n') 
stopWords.append('\n')

In [ ]:
import jieba
sentence=[]
sentence_test=[]

#透過jieba分詞工具，分別處理train和test資料
for content in train:
    _sentence=list(jieba.cut(content, cut_all=True))
    sentence.append(_sentence)
for content in test:
    _sentence=list(jieba.cut(content, cut_all=True))
    sentence_test.append(_sentence)

remainderWords2 = []
remainderWords_test = []

#將斷詞分別從train和test資料中移除
for content in sentence:
    remainderWords2.append(list(filter(lambda a: a not in stopWords, content)))
for content in sentence_test:
    remainderWords_test.append(list(filter(lambda a: a not in stopWords, content)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.810 seconds.
Prefix dict has been built successfully.


In [ ]:
print(train[:2])

('好，爱吃夹油条的煎饼果子', '一直订眉州东坡的外卖，很满意，特别是外送小哥，还帮我把门口的垃圾拿出去到垃圾桶，真得赞一个。')


In [ ]:
print(remainderWords2[:2])

[['爱', '吃', '夹', '油条', '煎饼', '煎饼果子', '果子'], ['订', '眉', '州', '东坡', '外卖', '满意', '特别', '外', '小哥', '还', '帮', '把门', '门口', '垃圾', '拿出', '垃圾', '垃圾桶', '真得', '赞', '一个']]


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
token = Tokenizer(num_words=3000)
token.fit_on_texts(remainderWords2)

In [ ]:
x_train_seq = token.texts_to_sequences(remainderWords2)
x_test_seq = token.texts_to_sequences(remainderWords_test)

In [ ]:
print(x_train_seq)

[[351, 5, 1033, 533, 57, 1034, 952], [55, 559, 597, 560, 24, 112, 37, 249, 43, 2, 803, 762, 168, 1427, 168, 1428, 2507, 32, 13], [5, 389, 399, 283], [141, 3], [400, 177, 15, 1429, 953, 1957, 871, 75], [476, 91, 58, 30, 1], [2508, 872, 284, 297, 633, 872, 457, 13, 954, 2509, 79, 2510, 2511, 368], [5, 310, 804, 54, 238, 9, 20, 285, 286, 1628, 1629, 7, 679, 194, 26, 78, 1630, 105, 1430, 194, 561, 10, 68, 274], [401, 173, 9, 402, 195], [92, 2, 679, 805, 141], [13, 6, 24, 318, 2512, 2, 2513, 1431, 458], [3, 141, 253, 61, 1631, 1632, 260, 8], [3, 4, 26, 2, 53], [46, 4, 3, 4], [112, 11, 178, 1633, 1958, 1634, 1959, 20, 44, 8, 1261, 806], [106, 1960, 1961, 61, 135, 120, 10, 234, 2514, 5, 399, 283, 2, 1432, 1635, 562, 390, 2515, 327, 336, 718, 807, 2516, 2517, 78], [140, 196, 261, 477, 1433, 122, 18, 56, 352, 337, 114, 16, 12, 3, 4, 1035, 2518, 5, 18, 155, 226, 109], [60, 1636, 23, 680, 279, 6], [92, 563, 158, 1127, 183, 25, 214, 504, 2519, 85, 218, 564, 634, 14], [7, 238, 72, 100, 8, 1262, 230

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=50)
x_test = sequence.pad_sequences(x_test_seq, maxlen=50)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding

In [ ]:
model = Sequential()

model.add(Embedding(output_dim=128, input_dim=3000, input_length=50))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(units=256, activation='relu' ))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           384000    
                                                                 
 dropout (Dropout)           (None, 50, 128)           0         
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 256)               1638656   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,022,913
Trainable params: 2,022,913
Non-

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
              
train_history =model.fit(x_train, 
                         labels_train,
                         batch_size=100, 
                         epochs=10,
                         verbose=2,
                         validation_split=0.2)

Epoch 1/10
52/52 - 3s - loss: 0.5934 - accuracy: 0.6715 - val_loss: 0.4477 - val_accuracy: 0.7859 - 3s/epoch - 60ms/step
Epoch 2/10
52/52 - 2s - loss: 0.3290 - accuracy: 0.8652 - val_loss: 0.3545 - val_accuracy: 0.8578 - 2s/epoch - 40ms/step
Epoch 3/10
52/52 - 2s - loss: 0.2232 - accuracy: 0.9156 - val_loss: 0.3705 - val_accuracy: 0.8562 - 2s/epoch - 41ms/step
Epoch 4/10
52/52 - 3s - loss: 0.1563 - accuracy: 0.9449 - val_loss: 0.4314 - val_accuracy: 0.8453 - 3s/epoch - 52ms/step
Epoch 5/10
52/52 - 2s - loss: 0.1074 - accuracy: 0.9621 - val_loss: 0.4848 - val_accuracy: 0.8344 - 2s/epoch - 41ms/step
Epoch 6/10
52/52 - 2s - loss: 0.0806 - accuracy: 0.9732 - val_loss: 0.5420 - val_accuracy: 0.8375 - 2s/epoch - 40ms/step
Epoch 7/10
52/52 - 2s - loss: 0.0680 - accuracy: 0.9777 - val_loss: 0.5947 - val_accuracy: 0.8250 - 2s/epoch - 40ms/step
Epoch 8/10
52/52 - 2s - loss: 0.0558 - accuracy: 0.9805 - val_loss: 0.6385 - val_accuracy: 0.8219 - 2s/epoch - 40ms/step
Epoch 9/10
52/52 - 2s - loss: 0.

In [ ]:
scores = model.evaluate(x_test, labels_test, verbose=1)
scores[1]

50/50 [==============================] - 0s 5ms/step - loss: 0.7040 - accuracy: 0.8194


0.8193749785423279

In [ ]:
predict =(model.predict(x_test) > 0.5).astype("int32")
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve,auc
print('Area under test roc :',roc_auc_score(labels_test,predict))
print("Testing Accuracy :", accuracy_score(labels_test,predict))
print("Testing Precision :", precision_score(labels_test,predict))
print("Testing Recall :", recall_score(labels_test,predict))
print("Testing f1-score :", f1_score(labels_test,predict))


Area under test roc : 0.8193952888784718
Testing Accuracy : 0.819375
Testing Precision : 0.8154613466334164
Testing Recall : 0.8226415094339623
Testing f1-score : 0.8190356919223544


In [ ]:
def display_test_Sentiment(i):
    print(test[i])
    print('原始結果:',labels_test[i])
    print('預測結果:',predict[i])
display_test_Sentiment(0)

凉面一般额。。。。。。其他还行
原始結果: 0
預測結果: [0]


In [ ]:
from keras.layers import LSTM
# 建立模型
modelLSTM = Sequential()


modelLSTM.add(Embedding(output_dim=128, input_dim=3000 ,input_length=50))

# 隨機在神經網路中放棄20%的神經元，避免overfitting
modelLSTM.add(Dropout(0.5))

# 建立LSTM層 (32個神經元)
modelLSTM .add(LSTM(64))

# 建立隱藏層 (256個神經元)
modelLSTM .add(Dense(units= 256, activation='relu')) 

# 放棄20%的神經元，避免overfitting
modelLSTM .add(Dropout(0.5))

# 建立輸出層 (1個神經元)
modelLSTM .add(Dense(units=1,activation='sigmoid'))

# 查看模型摘要
modelLSTM .summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 128)           384000    
                                                                 
 dropout_2 (Dropout)         (None, 50, 128)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 256)               16640     
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 450,305
Trainable params: 450,305
Non-tr

In [ ]:
modelLSTM.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
              
train_history =modelLSTM.fit(x_train, 
                         labels_train,
                         batch_size=100, 
                         epochs=10,
                         verbose=2,
                         validation_split=0.2)

Epoch 1/10
52/52 - 10s - loss: 0.5989 - accuracy: 0.6891 - val_loss: 0.4510 - val_accuracy: 0.7906 - 10s/epoch - 188ms/step
Epoch 2/10
52/52 - 8s - loss: 0.3301 - accuracy: 0.8656 - val_loss: 0.3639 - val_accuracy: 0.8508 - 8s/epoch - 152ms/step
Epoch 3/10
52/52 - 5s - loss: 0.2617 - accuracy: 0.9000 - val_loss: 0.3693 - val_accuracy: 0.8539 - 5s/epoch - 93ms/step
Epoch 4/10
52/52 - 5s - loss: 0.2263 - accuracy: 0.9189 - val_loss: 0.3923 - val_accuracy: 0.8547 - 5s/epoch - 93ms/step
Epoch 5/10
52/52 - 5s - loss: 0.1971 - accuracy: 0.9322 - val_loss: 0.4187 - val_accuracy: 0.8555 - 5s/epoch - 96ms/step
Epoch 6/10
52/52 - 5s - loss: 0.1821 - accuracy: 0.9350 - val_loss: 0.4424 - val_accuracy: 0.8477 - 5s/epoch - 93ms/step
Epoch 7/10
52/52 - 5s - loss: 0.1681 - accuracy: 0.9387 - val_loss: 0.4701 - val_accuracy: 0.8453 - 5s/epoch - 102ms/step
Epoch 8/10
52/52 - 5s - loss: 0.1543 - accuracy: 0.9424 - val_loss: 0.5101 - val_accuracy: 0.8406 - 5s/epoch - 93ms/step
Epoch 9/10
52/52 - 5s - los

In [ ]:
predict =(modelLSTM.predict(x_test) > 0.5).astype("int32")
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve,auc
print('Area under test roc :',roc_auc_score(labels_test,predict))
print("Testing Accuracy :", accuracy_score(labels_test,predict))
print("Testing Precision :", precision_score(labels_test,predict))
print("Testing Recall :", recall_score(labels_test,predict))
print("Testing f1-score :", f1_score(labels_test,predict))

  

Area under test roc : 0.8417320989101137
Testing Accuracy : 0.841875
Testing Precision : 0.8565789473684211
Testing Recall : 0.8188679245283019
Testing f1-score : 0.837299035369775


In [ ]:
scores = modelLSTM.evaluate(x_test, labels_test, verbose=1)
scores[1]

50/50 [==============================] - 1s 11ms/step - loss: 0.5599 - accuracy: 0.8419


0.8418750166893005

In [ ]:
def display_test_Sentiment(i):
    print(test[i])
    print('原始結果:',labels_test[i])
    print('預測結果:',predict[i])

In [ ]:
display_test_Sentiment(0)

凉面一般额。。。。。。其他还行
原始結果: 0
預測結果: [0]


In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Embedding
# 建立模型
modelCNN = Sequential()

# 輸出的維度是32,希望將數字list轉為32維度的向量
# 輸入的維度是3800,也就是我們之前建立的字典3800字
# 數字list截長補短後都是380個數字
modelCNN.add(Embedding(output_dim=32, input_dim=3800, input_length=50))

# 隨機在神經網路中放棄20%的神經元，避免overfitting
modelCNN.add(Dropout(0.2))

# 建立卷積層1(Convolution1)
# 隨機產生32個filter weight
# 濾鏡大小3
modelCNN.add(Conv1D(filters=32,padding='same', activation='relu', kernel_size=3))
modelCNN .add(Dropout(0.2))

# 建立池化層1(Pooling1)
modelCNN.add(MaxPooling1D(pool_size=2))


# 建立卷積層2(Convolution2)
modelCNN.add(Conv1D(filters=64,padding='same', activation='relu', kernel_size=4))
modelCNN .add(Dropout(0.2))

# 建立池化層2(Pooling2)
modelCNN.add(MaxPooling1D(pool_size=3))


# 建立平坦層
modelCNN.add(Flatten())
modelCNN.add(Dropout(0.2))

# 建立隱藏層 (250個神經元)
modelCNN.add(Dense(250, activation='relu'))
modelCNN.add(Dropout(0.2))

# 建立輸出層 (1個神經元)
modelCNN.add(Dense(1, activation='sigmoid'))

# 查看模型摘要
modelCNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 32)            121600    
                                                                 
 dropout_4 (Dropout)         (None, 50, 32)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 32)            3104      
                                                                 
 dropout_5 (Dropout)         (None, 50, 32)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 25, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 64)            8256      
                                                      

In [ ]:
modelCNN.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
              
train_history =modelCNN.fit(x_train, 
                         labels_train,
                         batch_size=100, 
                         epochs=10,
                         verbose=2,
                         validation_split=0.2)

Epoch 1/10
52/52 - 4s - loss: 0.6436 - accuracy: 0.6299 - val_loss: 0.5764 - val_accuracy: 0.7023 - 4s/epoch - 73ms/step
Epoch 2/10
52/52 - 2s - loss: 0.3974 - accuracy: 0.8217 - val_loss: 0.3861 - val_accuracy: 0.8391 - 2s/epoch - 30ms/step
Epoch 3/10
52/52 - 1s - loss: 0.2619 - accuracy: 0.9008 - val_loss: 0.3581 - val_accuracy: 0.8578 - 1s/epoch - 25ms/step
Epoch 4/10
52/52 - 1s - loss: 0.2160 - accuracy: 0.9217 - val_loss: 0.3694 - val_accuracy: 0.8531 - 1s/epoch - 25ms/step
Epoch 5/10
52/52 - 1s - loss: 0.1838 - accuracy: 0.9359 - val_loss: 0.3952 - val_accuracy: 0.8461 - 1s/epoch - 25ms/step
Epoch 6/10
52/52 - 1s - loss: 0.1676 - accuracy: 0.9389 - val_loss: 0.4003 - val_accuracy: 0.8453 - 1s/epoch - 25ms/step
Epoch 7/10
52/52 - 1s - loss: 0.1518 - accuracy: 0.9451 - val_loss: 0.4429 - val_accuracy: 0.8367 - 1s/epoch - 25ms/step
Epoch 8/10
52/52 - 1s - loss: 0.1265 - accuracy: 0.9572 - val_loss: 0.4636 - val_accuracy: 0.8383 - 1s/epoch - 25ms/step
Epoch 9/10
52/52 - 1s - loss: 0.

In [ ]:
predict =(modelCNN.predict(x_test) > 0.5).astype("int32")
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve,auc
print('Area under test roc :',roc_auc_score(labels_test,predict))
print("Testing Accuracy :", accuracy_score(labels_test,predict))
print("Testing Precision :", precision_score(labels_test,predict))
print("Testing Recall :", recall_score(labels_test,predict))
print("Testing f1-score :", f1_score(labels_test,predict))


Area under test roc : 0.840513301300832
Testing Accuracy : 0.840625
Testing Precision : 0.8515625
Testing Recall : 0.8226415094339623
Testing f1-score : 0.836852207293666


In [ ]:
predict=modelCNN.predict(x_test)
predict = np.argmax(predict,axis=1)
def display_test_Sentiment(i):
    print(test[i])
    print('原始結果:',labels_test[i])
    print('預測結果:',predict[i])

In [ ]:
scores = modelCNN.evaluate(x_test, labels_test, verbose=1)
scores[1]

50/50 [==============================] - 0s 4ms/step - loss: 0.4955 - accuracy: 0.8406


0.840624988079071

In [ ]:
display_test_Sentiment(0)

凉面一般额。。。。。。其他还行
原始結果: 0
預測結果: 0
